# [STARTER] Exercise - Building an Agentic RAG System

In this exercise, you will build an Agentic RAG (Retrieval-Augmented Generation) system that 
combines the power of AI agents with traditional RAG pipelines. You'll create an agent that 
can decide when and how to retrieve information from different sources, including vector 
databases, web search, and other tools.


## Challenge

Your challenge is to create an Agentic RAG system that can:

- Build a RAG pipeline as a tool that can be used by the agent
- Create an agent that can decide which tool to use based on the query
- Handle different types of queries intelligently
- Combine information from multiple sources when needed


## Setup
First, let's import the necessary libraries:

In [2]:
# Only needed for Udacity workspace

import importlib.util
import sys

# Check if 'pysqlite3' is available before importing
if importlib.util.find_spec("pysqlite3") is not None:
    import pysqlite3
    sys.modules['sqlite3'] = sys.modules.pop('pysqlite3')

In [3]:
import os
from typing import List
from dotenv import load_dotenv

from lib.agents import Agent
from lib.llm import LLM
from lib.state_machine import Run
from lib.messages import BaseMessage
from lib.tooling import tool
from lib.vector_db import VectorStoreManager, CorpusLoaderService
from lib.rag import RAG

In [4]:
load_dotenv()

True

In [5]:
OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")

## Load data to Vector DB

In [6]:
db = VectorStoreManager(OPENAI_API_KEY)
db

VectorStoreManager():<chromadb.api.client.Client object at 0x000002652CB24440>

In [7]:
loader_service = CorpusLoaderService(db)

In [8]:
rag_llm = LLM(
    model="gpt-4o-mini",
    temperature=0.3,
)

In [9]:
# TODO: Add the games pdf file path with the extenstion .pdf
# And define a store name in load_pdf() method

games_market_rag = RAG(
    llm=rag_llm,
    vector_store = loader_service.load_pdf(
        store_name="games_market",
        pdf_path="TheGamingIndustry2024.pdf"
    )
)

VectorStore `games_market` ready!
Pages from `TheGamingIndustry2024.pdf` added!


In [10]:
result:Run = games_market_rag.invoke(
    "What's the  state of virtual reality"
)
print(result.get_final_state()["answer"])

[StateMachine] Starting: __entry__
[StateMachine] Executing step: retrieve
[StateMachine] Executing step: augment
[StateMachine] Executing step: generate
[StateMachine] Terminating: __termination__
The state of virtual reality (VR) in 2024 is characterized by significant advancements and immersive gaming experiences. VR technology allows gamers to explore expansive digital worlds and engage in interactive environments that feel realistic. Notable examples include games like "Half-Life: Alyx," which showcase high-fidelity graphics, intuitive controls, and immersive audio, enhancing the overall gaming experience. Additionally, VR is being utilized beyond entertainment, with applications in education, cognitive therapy, and physical rehabilitation, indicating its transformative potential across various fields. Overall, VR is playing a crucial role in reshaping the gaming landscape and expanding its influence.


In [11]:
# TODO: Add the electric vehicles pdf file path with the extenstion .pdf
# And define a store name in load_pdf() method

electric_vehicles_rag = RAG(
    llm=rag_llm,
    vector_store = loader_service.load_pdf(
        store_name="electric_vehicles",
        pdf_path="GlobalEVOutlook2025.pdf"
    )
)

VectorStore `electric_vehicles` ready!
Pages from `GlobalEVOutlook2025.pdf` added!


In [12]:
result:Run = electric_vehicles_rag.invoke("What was the number of electric car sales and their market share in Brazil in 2024?")
print(result.get_final_state()["answer"])

[StateMachine] Starting: __entry__
[StateMachine] Executing step: retrieve
[StateMachine] Executing step: augment
[StateMachine] Executing step: generate
[StateMachine] Terminating: __termination__
In 2024, Brazil had nearly 125,000 electric car sales, which represented a market share of 6.5%.


## Tools

In a simple form, Agentic RAG can act like a router, choosing between multiple external sources to retrieve relevant information. These sources aren't limited to databases, they can also include tools like web search or APIs for services such as Slack or email.

In this case it will choose between two collections.

In [13]:
# TODO: Define a tool that returns result.get_final_state()["answer"]
# DONOT Forget about defining the tool docstrings
@tool
def search_global_ev_collection(query):
    """
        Search the vector database for relevant information.
    
        Document Name: Global EV Outlook 2025
        Published date: May 2025
        Publisher: International Energy Agency (IEA)

        ```
        The Global EV Outlook is an annual publication that reports on recent
        developments in electric mobility around the world. It is developed with the support
        of members of the Electric Vehicles Initiative (EVI).
        ```
    Args:
        query (str): The search query.
    
    """
    result:Run = electric_vehicles_rag.invoke(query)
    return result.get_final_state()["answer"]

In [24]:
# TODO: Define a tool that returns result.get_final_state()["answer"]
# DONOT Forget about defining the tool docstrings
@tool
def search_games_market_report_collection(query):
    """
        Search the vector database for relevant information.
    
        Document Name: The Gaming Industry in 2024: Trends, Technologies & Predictions
        Published date: 2024
        Publisher: IXIE Gaming

        ```
        The Gaming Industry 2024 report provides insights into the latest trends,
        market dynamics, and growth opportunities within the global gaming sector.
        ```
    Args:
        query (str): The search query."""
    
    result:Run = games_market_rag.invoke(query)
    return result.get_final_state()["answer"]

In [25]:
# TODO: Add the tools you have defined and the instructions to your agent
agentic_rag = Agent(
    model_name="gpt-4o-mini",
    tools=[search_games_market_report_collection, search_global_ev_collection],    
    instructions=(
            "You are an Agentic RAG assistant that can intelligently decide which tools to use "
            "to answer user questions. Reason about about the response, change the query and call the tool again if needed "
            "in order to get better results. Always explain your reasoning for tool selection and provide comprehensive answers."
           )
)

In [26]:
def print_messages(messages: List[BaseMessage]):
    for m in messages: 
        print(f" -> (role = {m.role}, content = {m.content}, tool_calls = {getattr(m, 'tool_calls', None)})")

## Run

In [17]:
run_1 = agentic_rag.invoke(
    query="Who won the 2025 Oscar for International Movie?", 
    session_id="oscar",
)

print("\nMessages from run 1:")
messages = run_1.get_final_state()["messages"]
print_messages(messages)

[StateMachine] Starting: __entry__
[StateMachine] Executing step: message_prep
[StateMachine] Executing step: llm_processor
[StateMachine] Terminating: __termination__

Messages from run 1:
 -> (role = system, content = 
            You are a helpful RAG Agent that can decide wisely which tool to use.
            Answer user questions using ONLY information retrieved from the available tools.
            Choose the correct tool based on the topic (gaming industry or electric vehicles), and use both if the question spans both domains.
            If the retrieved information is insufficient, clearly say so and do not invent details.
            , tool_calls = None)
 -> (role = user, content = Who won the 2025 Oscar for International Movie?, tool_calls = None)
 -> (role = assistant, content = I don't have information on the 2025 Oscar winners, including the International Movie category. You may need to check a reliable entertainment news source for the latest updates on the Oscars., tool

In [21]:
run_2 = agentic_rag.invoke(
    query= (
        "Which two countries accounted for most of the electric car exports from " 
        "the Asia Pacific region (excluding China) in 2024?"
    ),
    session_id="electric_car",
)

print("\nMessages from run 2:")
messages = run_2.get_final_state()["messages"]
print_messages(messages)

[StateMachine] Starting: __entry__
[StateMachine] Executing step: message_prep
[StateMachine] Executing step: llm_processor
[StateMachine] Starting: __entry__
[StateMachine] Executing step: retrieve
[StateMachine] Executing step: augment
[StateMachine] Executing step: generate
[StateMachine] Terminating: __termination__
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Terminating: __termination__

Messages from run 2:
 -> (role = system, content = 
            "You are an Agentic RAG assistant that can intelligently decide which tools to use "
        "to answer user questions. Reason about about the response, change the query and call the tool again if needed "
        "in order to get better results. Always explain your reasoning for tool selection and provide comprehensive answers."
            , tool_calls = None)
 -> (role = user, content = Which two countries accounted for most of the electric car exports from the Asia Pacif

In [27]:
run_3 = agentic_rag.invoke(
    query= (
        "Why is generative AI seen more as an accelerator than a replacement in game development?"
    ),
    session_id="games",
)

print("\nMessages from run 3:")
messages = run_3.get_final_state()["messages"]
print_messages(messages)

[StateMachine] Starting: __entry__
[StateMachine] Executing step: message_prep
[StateMachine] Executing step: llm_processor
[StateMachine] Starting: __entry__
[StateMachine] Executing step: retrieve
[StateMachine] Executing step: augment
[StateMachine] Executing step: generate
[StateMachine] Terminating: __termination__
[StateMachine] Executing step: tool_executor
[StateMachine] Executing step: llm_processor
[StateMachine] Terminating: __termination__

Messages from run 3:
 -> (role = system, content = You are an Agentic RAG assistant that can intelligently decide which tools to use to answer user questions. Reason about about the response, change the query and call the tool again if needed in order to get better results. Always explain your reasoning for tool selection and provide comprehensive answers., tool_calls = None)
 -> (role = user, content = Why is generative AI seen more as an accelerator than a replacement in game development?, tool_calls = None)
 -> (role = assistant, cont